In [1]:
# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb

In [2]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import MODELS_FOLDER

In [3]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
import libs.model_archs
import libs.utils
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [5]:
# define dataset and model architecture
dataset = "cifar10"
# Select ResNet Version
resnet_version = 1
model_arch = "resnetA" + str(resnet_version)

# set training hyperparameters
batch_size = 128
n_epochs = 200

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
class CustomImageDataGenerator(ImageDataGenerator):
    def __init__(self, cutout_mask_size = 0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cutout_mask_size = cutout_mask_size
        
    def cutout(self, x, y):
        return np.array(list(map(self._cutout, x))), y
    
    def _cutout(self, image_origin):
        # 最後に使うfill()は元の画像を書き換えるので、コピーしておく
        image = np.copy(image_origin)
        mask_value = image.mean()

        h, w, _ = image.shape
        # マスクをかける場所のtop, leftをランダムに決める
        # はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
        top = np.random.randint(0 - self.cutout_mask_size // 2, h - self.cutout_mask_size)
        left = np.random.randint(0 - self.cutout_mask_size // 2, w - self.cutout_mask_size)
        bottom = top + self.cutout_mask_size
        right = left + self.cutout_mask_size

        # はみ出した場合の処理
        if top < 0:
            top = 0
        if left < 0:
            left = 0

        # マスク部分の画素値を平均値で埋める
        image[top:bottom, left:right, :].fill(mask_value)
        return image
    
    def flow(self, *args, **kwargs):
        batches = super().flow(*args, **kwargs)
        
        # 拡張処理
        while True:
            batch_x, batch_y = next(batches)
            
            if self.cutout_mask_size > 0:
                result = self.cutout(batch_x, batch_y)
                batch_x, batch_y = result                        
                
            yield (batch_x, batch_y)     

datagen_parameters = {"horizontal_flip": True, "width_shift_range": 0.1, "height_shift_range": 0.1, "cutout_mask_size": 16}
datagen = CustomImageDataGenerator(**datagen_parameters)
datagen_for_test = ImageDataGenerator()
# ZCA whiteningなどを行う場合が以下の実行が必要
# datagen.fit(train_x)
# datagen_for_test.fit(test_x)

In [7]:
# global seed
seed = model_seeds[0]
tf.random.set_seed(seed)
np.random.seed(seed)

# prepare data
# dataset_loader = getattr(libs.utils, 'prepare_'+dataset)
# to_categorical=False
# (x_train, y_train), (x_test, y_test) = dataset_loader(to_categorical)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

dataset_test = (x_test, y_test) 
dataset_train = (x_train, y_train)

x_train = dataset_train[0] / 255
y_train = dataset_train[1].squeeze()
x_test = dataset_test[0] / 255
y_test = dataset_test[1].squeeze()

number_train = x_train.shape[0]
number_test = x_test.shape[0]

In [8]:
from tensorflow.keras.callbacks import Callback
class LearningController(Callback):
    def __init__(self, num_epoch=0, learn_minute=0):
        self.num_epoch = num_epoch
        self.learn_second = learn_minute * 60
        if self.learn_second > 0:
            print("Learning rate is controled by time.")
        elif self.num_epoch > 0:
            print("Learning rate is controled by epoch.")
        
    def on_train_begin(self, logs=None):
        if self.learn_second > 0:
            self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        if self.learn_second > 0:
            current_time = time.time()
            if current_time - self.start_time > self.learn_second:
                self.model.stop_training = True
                print("Time is up.")
                return

            if current_time - self.start_time > self.learn_second / 2:
                self.model.optimizer.lr = lr * 0.1            
            if current_time - self.start_time > self.learn_second * 3 / 4:
                self.model.optimizer.lr = lr * 0.01
                
        elif self.num_epoch > 0:
            if epoch > self.num_epoch / 2:
                self.model.optimizer.lr = lr * 0.1            
            if epoch > self.num_epoch * 3 / 4:
                self.model.optimizer.lr = lr * 0.01
                    
        print('\nlr:%.2e' % self.model.optimizer.lr.value())

In [9]:
# create model
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D, Add, Input, Flatten
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2


input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
n_classes = y_train.shape

# n_channels = x_train.shape[3] # no. of channels (?)
# # Computed depth of model
# if resnet_version == 1:
# 	depth = n_channels * 6 + 2
# elif resnet_version == 2:
# 	depth = n_channels * 9 + 2

# model_generator = getattr(libs.model_archs, model_arch)
# model = model_generator(input_shape, depth, n_classes)

# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb
n = 9 # 56 layers
channels = [16, 32, 64]

inputs = Input(shape=input_shape)
x = Conv2D(channels[0], 
           kernel_size=(3, 3), 
           padding="same", 
           kernel_initializer="he_normal", 
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation(tf.nn.relu)(x)

for c in channels:
    for i in range(n):
        subsampling = i == 0 and c > 16
        strides = (2, 2) if subsampling else (1, 1)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   strides=strides, 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation(tf.nn.relu)(y)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if subsampling:
            x = Conv2D(c, 
                       kernel_size=(1, 1), 
                       strides=(2, 2), 
                       padding="same", 
                       kernel_initializer="he_normal", 
                       kernel_regularizer=l2(1e-4))(x)
        x = Add()([x, y])
        x = Activation(tf.nn.relu)(x)

x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
outputs = Dense(10, 
                activation=tf.nn.softmax, 
                kernel_initializer="he_normal")(x)

model = Model(inputs=inputs, outputs=outputs)

model._name = "resnet_lrs_checkpnt_cutout" + str(6 * n + 2)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "ResNet-for-CIFAR-10-with-Keras.h5", 
                             monitor="val_loss", 
                             verbose=1, 
                             save_best_only=True)

In [11]:
# compile model
from tensorflow.keras.optimizers import SGD
# model.compile(loss ='categorical_crossentropy',
#               optimizer ='adam',
#               metrics =['accuracy'])
lr = 0.1
optimizer = SGD(learning_rate=lr, momentum=0.9)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

learning_controller = LearningController(n_epochs)
callbacks = [checkpoint, learning_controller]
# train model
model.fit(datagen.flow(x_train, 
                         y_train, 
                         batch_size=batch_size), 
          epochs=n_epochs,
          steps_per_epoch=number_train//batch_size,
          validation_data=datagen_for_test.flow(x_test, 
                                                  y_test, 
                                                  batch_size=batch_size),
          validation_steps=number_test//batch_size,
          validation_split=0.2,
          callbacks=callbacks)

Learning rate is controled by epoch.
Epoch 1/200
390/390 [==============================] - ETA: 0s - loss: 3.2654 - accuracy: 0.1150
Epoch 1: val_loss improved from inf to 2.91118, saving model to ResNet-for-CIFAR-10-with-Keras.h5

lr:1.00e-01
390/390 [==============================] - 68s 106ms/step - loss: 3.2654 - accuracy: 0.1150 - val_loss: 2.9112 - val_accuracy: 0.1492
Epoch 2/200
390/390 [==============================] - ETA: 0s - loss: 2.6442 - accuracy: 0.2281
Epoch 2: val_loss did not improve from 2.91118

lr:1.00e-01
390/390 [==============================] - 40s 102ms/step - loss: 2.6442 - accuracy: 0.2281 - val_loss: 2.9821 - val_accuracy: 0.1978
Epoch 3/200
390/390 [==============================] - ETA: 0s - loss: 2.3322 - accuracy: 0.3251
Epoch 3: val_loss improved from 2.91118 to 2.34897, saving model to ResNet-for-CIFAR-10-with-Keras.h5

lr:1.00e-01
390/390 [==============================] - 40s 104ms/step - loss: 2.3322 - accuracy: 0.3251 - val_loss: 2.3490 - val_a

In [12]:
# evaluate model
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=batch_size)

print("Original Accuracy: ",score)

79/79 [==============================] - 3s 38ms/step - loss: 0.4225 - accuracy: 0.9263
Original Accuracy:  [0.4224640727043152, 0.9262999892234802]


In [13]:
# save model
model_type = dataset + "--" + model_arch
model_instance = model_type + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model.name)
pathlib.Path(model_subdir).mkdir(parents=True, exist_ok=True)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model.save(model_file)

In [14]:
model.summary()

Model: "resnet_lrs_checkpnt_cutout56"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batc